# Retrieve models, data, libraries etc.



In [1]:
import os
import sys
import copy
import random
import json
import yaml
import glob
import cv2
import numpy as np
import time
import csv
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import requests
from   zipfile import ZipFile
%matplotlib inline
import pandas as pd
import logging
import torch

# Set up logging
logging.basicConfig(level=logging.INFO)


In [7]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True) # Everything is connected to mye drive now (will find a better way)

DATA_DIR = "/content/drive/MyDrive/FISHIAL"

Mounted at /content/drive/


In [3]:
def download_and_unzip(url, save_path, extract_dir):
    print("Downloading assets...")
    file = requests.get(url)

    open(save_path, "wb").write(file.content)
    print("Download completed.")

    try:
        if save_path.endswith(".zip"):
            with ZipFile(save_path, 'r') as zip_ref:
                zip_ref.extractall(extract_dir)
            print("Extraction Done")
    except Exception as e:
        print(f"An error occurred: {e}")

def download_image(url):
    filename = os.path.basename(url)

    response = requests.get(url)
    response.raise_for_status()

    with open(filename, 'wb') as file:
        file.write(response.content)

    return os.path.abspath(filename)

def get_basename(path):
  return os.path.basename(path)

def print_fish_data(fish_data):
    for idx, fish in enumerate(fish_data, start=1):
        print(f"ID: {idx}")
        print(f"Name: {fish['name']}")
        print(f"Species ID: {fish['species_id']}")
        print(f"Distance: {fish['distance']:.3f}")
        print(f"Accuracy: {fish['accuracy']:.2%}")
        print("-" * 40)


In [5]:
# Links to models
MODEL_URLS = {
    'classification': 'https://storage.googleapis.com/fishial-ml-resources/classification_rectangle_v7-1.zip',
    'segmentation': 'https://storage.googleapis.com/fishial-ml-resources/segmentator_fpn_res18_416_1.zip',
    'detection': 'https://storage.googleapis.com/fishial-ml-resources/detector_v10_m3.zip',
    'face': 'https://storage.googleapis.com/fishial-ml-resources/face_yolo.zip'
}

# Model directories
MODEL_DIRS = {
    'classification': "models/classification",
    'segmentation': "models/segmentation",
    'detection': "models/detection",
    'face': "models/face_detector"
}

# Create directories and download models
for model_name, url in MODEL_URLS.items():
    model_dir = MODEL_DIRS[model_name]
    zip_path = os.path.join(os.getcwd(), get_basename(url))

    os.makedirs(model_dir, exist_ok=True)  # Create directory if it doesn't exist
    download_and_unzip(url, zip_path, model_dir)  # Download and unzip the model

  # Remove the zip file after extraction
    try:
        os.remove(zip_path)
        logging.info(f"Removed zip file {zip_path}")
    except Exception as e:
        logging.error(f"Failed to remove zip file {zip_path}: {e}")

Download completed.
Extraction Done
Download completed.
Extraction Done
Download completed.
Extraction Done
Download completed.
Extraction Done


In [6]:
from models.classification.inference import EmbeddingClassifier
from models.detection.inference import YOLOInference
from models.segmentation.inference import Inference
from models.face_detector.inference import YOLOInference as FaceInference


# Model initialization
classifier = EmbeddingClassifier(
    os.path.join(MODEL_DIRS['classification'], 'model.ts'),
    os.path.join(MODEL_DIRS['classification'], 'database.pt')
)

segmentator = Inference(
    model_path=os.path.join(MODEL_DIRS['segmentation'], 'model.ts'),
    image_size=416
)

detector = YOLOInference(
    os.path.join(MODEL_DIRS['detection'], 'model.ts'),
    imsz=(640, 640),
    conf_threshold=0.9,
    nms_threshold=0.3,
    yolo_ver='v10'
)

face_detector = FaceInference(
    os.path.join(MODEL_DIRS['face'], 'model.ts'),
    imsz=(640, 640),
    conf_threshold=0.69,
    nms_threshold=0.5,
    yolo_ver='v8'
)

# Classification


In [9]:
image_folder_path = os.path.join(DATA_DIR, "timor_leste") # Images
fishial_csv = os.path.join(DATA_DIR, "fishial_V7.csv") # Fishial's species for V7
annotation_data_csv = os.path.join(DATA_DIR, "timor-leste.csv") # Annotation info for ground truth

# Read species list
fishial_species = pd.read_csv(fishial_csv, encoding="utf-8-sig", header=None, sep=";")
fishial_species_latin = set(fishial_species[0].str.strip())

# Read annotation info
annotation_info = pd.read_csv(annotation_data_csv, encoding="utf-8-sig", header=None)
annotation_info = annotation_info[[2, 4]].rename(columns={2: "filename", 4: "species"})

# Keep only valid rows (species in fishial list)
valid_rows = annotation_info[annotation_info["species"].isin(fishial_species_latin)]

# Filter for files that actually exist
existing_files = set(os.listdir(image_folder_path))
valid_rows = valid_rows[valid_rows["filename"].isin(existing_files)]

# Convert to list
file_names = valid_rows["filename"].tolist()

print("Images:", file_names)
print("Number of images:", len(file_names))


Images: ['1689120590238.jpg', '1689577829534.jpg', '1689577974563.jpg', '1689594468820.jpg', '1689651076768.jpg', '1689651811316.jpg', '1689652491689.jpg', '1689733035992.jpg', '1689739881340.jpg', '1689739901296.jpg', '1689740066618.jpg', '1689740260887.jpg', '1689743257346.jpg', '1690337039741.jpg', '1690349481994.jpg', '1690349481994.jpg', '1690466885625.jpg', '1690544392983.jpg', '1690544392983.jpg', '1690598682706.jpg', '1690599089156.jpg', '1690599089156.jpg', '1690890877921.jpg', '1691197797009.jpg', '1691197797009.jpg', '1691201510682.jpg', '1691201635655.jpg', '1691201728522.jpg', '1691234458029.jpg', '1691235083249.jpg', '1691235144639.jpg', '1691235476798.jpg', '1691235509689.jpg', '1691235988859.jpg', '1691236115459.jpg', '1692330985857.jpg', '1692413225981.jpg', '1692413842219.jpg', '1692414093710.jpg', '1692414486273.jpg', '1692581137223.jpg', '1692581157271.jpg', '1693045597639.jpg', '1693046054194.jpg', '1693046054194.jpg', '1693046250037.jpg', '1693046250037.jpg', '169

In [11]:

for img_path in file_names:
    try:
        print("\n" + "=" * 60)
        print(f"Processing: {os.path.basename(img_path)}")

        fish_bgr_np = cv2.imread(os.path.join(DATA_DIR, "timor_leste",img_path))
        if fish_bgr_np is None:
            print("Could not read image, skipping.")
            continue

        visulize_img_bgr = fish_bgr_np.copy()
        visulize_img_rgb = cv2.cvtColor(fish_bgr_np, cv2.COLOR_BGR2RGB)
        visulize_img = copy.deepcopy(visulize_img_rgb)

        # ---- OBJECT DETECTION ----
        boxes = detector.predict(visulize_img_rgb)[0]

        # Track if we created any mask preview for saving
        mask_idx = 0
        for box in boxes:
            cropped_fish_bgr = box.get_mask_BGR()
            cropped_fish_rgb = box.get_mask_RGB()

            # ---- SEGMENTATION ----
            segmented_polygons = segmentator.predict(cropped_fish_bgr)[0]
            cropped_fish_mask = segmented_polygons.mask_polygon(cropped_fish_rgb)

            # draw polygon on the full image
            segmented_polygons.move_to(box.x1, box.y1)
            segmented_polygons.draw_polygon(visulize_img)

            # ---- CLASSIFICATION ----
            classification_result = classifier.batch_inference([cropped_fish_bgr])[0]
            label = (
                f"{classification_result[0]['name']} | {round(classification_result[0]['accuracy'], 3)}"
                if len(classification_result)
                else "Not Found"
            )

            box.draw_label(visulize_img, label)
            box.draw_box(visulize_img)

            # Show each cropped mask preview
            print("-" * 50)
            print_fish_data(classification_result)

            plt.figure()
            plt.imshow(cropped_fish_mask)
            plt.axis('off')
            plt.title(f"{os.path.basename(img_path)} - crop {mask_idx}")
            plt.show()

        # Show the final visualization for the image
        plt.figure()
        plt.imshow(visulize_img)
        plt.axis('off')
        plt.title(os.path.basename(img_path))
        plt.show()

    except Exception as e:
        print(f"Error processing {img_path}: {e}")

Output hidden; open in https://colab.research.google.com to view.

In [13]:
def norm(name):
    return os.path.basename(str(name)).strip().lower()

# A case-insensitive lookup of actual image file paths in timor-leste (handles subfolders + .JPG/.jpg)
files_ci = {}
for root, _, files in os.walk(image_folder_path):
    for f in files:
        files_ci[f.lower()] = os.path.join(root, f)

csv_lookup = {}

for _, row in annotation_info.iterrows():
    try:
        img_name = norm(row["filename"])  # normalized filename (column 3 originally)
        species = str(row["species"]).strip() if pd.notna(row["species"]) else None  # column 5 originally
        if img_name:
            csv_lookup[img_name] = species
    except Exception:
        continue  # skip malformed rows

results = []
num_found = 0
num_correct = 0

for img_name in file_names:
    key = norm(img_name)
    gt = csv_lookup.get(key, None)  # ground-truth species from CSV

    img_path = files_ci.get(key, None)
    if not img_path or not os.path.exists(img_path):
        print(f"Could not locate on disk: {img_name} — skipping.")
        results.append((img_name, gt, None, None, "missing_file"))
        continue

    # Read image
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        print(f"Could not read: {img_path} — skipping.")
        results.append((img_name, gt, None, None, "read_error"))
        continue
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # Predict
    try:
        boxes = detector.predict(img_rgb)[0]
    except Exception as e:
        print(f"Detector failed on {img_name}: {e}")
        results.append((img_name, gt, None, None, "detector_fail"))
        continue

    best_pred, best_acc = None, -1.0
    for box in boxes:
        try:
            crop_bgr = box.get_mask_BGR()
            cls_out = classifier.batch_inference([crop_bgr])[0]
            if cls_out:
                top = cls_out[0]  # assume sorted by confidence
                name = top.get("name")
                acc = float(top.get("accuracy", 0.0))
                if acc > best_acc:
                    best_acc = acc
                    best_pred = name
        except Exception:
            # skip this box if classification fails
            pass

    # Compare with ground truth
    if gt is not None:
        num_found += 1
        is_correct = (best_pred == gt) if best_pred is not None else False
        if is_correct:
            num_correct += 1
            status = "MATCH"
        else:
            status = "MISMATCH"

        results.append((img_name, gt, best_pred, best_acc, status))
        print(f"{img_name}: GT='{gt}' | PRED='{best_pred}' (acc={best_acc:.3f}) → {status}")
    else:
        status = "gt_not_found"
        results.append((img_name, None, best_pred, best_acc, status))
        print(f"{img_name}: GT not found in CSV | PRED='{best_pred}' (acc={best_acc:.3f})")

# Summary
if num_found > 0:
    overall_acc = num_correct / num_found
    print(f"\nFound GT for {num_found}/{len(file_names)} images. Top-1 accuracy: {overall_acc:.3f}")
else:
    print("\nNo matching filenames from the CSV were found.")

1689120590238.jpg: GT='Caranx sexfasciatus' | PRED='Seriola rivoliana' (acc=0.267) → MISMATCH
1689577829534.jpg: GT='Scomberomorus commerson' | PRED='None' (acc=-1.000) → MISMATCH
1689577974563.jpg: GT='Scomberomorus commerson' | PRED='Scomberomorus cavalla' (acc=0.933) → MISMATCH
1689594468820.jpg: GT='Lutjanus gibbus' | PRED='Stenotomus chrysops' (acc=0.867) → MISMATCH
1689651076768.jpg: GT='Scomberomorus commerson' | PRED='Scomberomorus cavalla' (acc=0.667) → MISMATCH
1689651811316.jpg: GT='Scomberomorus commerson' | PRED='Xiphias gladius' (acc=1.000) → MISMATCH
1689652491689.jpg: GT='Scomberomorus commerson' | PRED='None' (acc=-1.000) → MISMATCH
1689733035992.jpg: GT='Acanthocybium solandri' | PRED='Acanthocybium solandri' (acc=1.000) → MATCH
1689739881340.jpg: GT='Scomberomorus commerson' | PRED='None' (acc=-1.000) → MISMATCH
1689739901296.jpg: GT='Scomberomorus commerson' | PRED='Scomberomorus cavalla' (acc=0.933) → MISMATCH
1689740066618.jpg: GT='Scomberomorus commerson' | PRED=

In [14]:
results_df = pd.DataFrame(
    results,
    columns=["filename", "ground_truth", "prediction", "accuracy", "status"]
)

out_csv = os.path.join("/content/drive/MyDrive/FISHIAL", "evaluation_results_classification.csv")
os.makedirs(os.path.dirname(out_csv), exist_ok=True)

results_df.to_csv(out_csv, index=False, encoding="utf-8-sig")
print(f"\n Saved results to {out_csv}")



 Saved results to /content/drive/MyDrive/FISHIAL/evaluation_results_classification.csv


#Bounding box detection

In [15]:
# Code to make an array with all fishes from Timor Leste that exists in the Fishial data base
image_folder_path = os.path.join(DATA_DIR, "timor_leste") # Images
fishial_csv = os.path.join(DATA_DIR, "fishial_V7.csv") # Fishial's species for V7
annotation_data_csv = os.path.join(DATA_DIR, "timor-leste.csv") # Annotation info for ground truth

# Where to save results
save_dir = "/content/FISHIAL_outputs"
os.makedirs(save_dir, exist_ok=True)
all_files = sorted(os.listdir(image_folder_path))



In [17]:
detection_results = []  # to store (filename, num_detections)

for img_path in file_names:
    try:
        print("\n" + "=" * 60)
        print(f"Processing: {os.path.basename(img_path)}")

        full_path = os.path.join(DATA_DIR, "timor_leste", img_path)
        fish_bgr_np = cv2.imread(full_path)
        if fish_bgr_np is None:
            print("Could not read image, skipping.")
            detection_results.append((os.path.basename(img_path), 0))
            continue

        visulize_img_rgb = cv2.cvtColor(fish_bgr_np, cv2.COLOR_BGR2RGB)
        visulize_img = visulize_img_rgb.copy()

        # ---- OBJECT DETECTION (ONLY) ----
        boxes = detector.predict(visulize_img_rgb)[0]
        num_detections = len(boxes)

        # Save results
        detection_results.append((os.path.basename(img_path), num_detections))

        # Draw bounding boxes
        for box in boxes:
            box.draw_box(visulize_img)

        # Show + save visualization
        plt.figure()
        plt.imshow(visulize_img)
        plt.axis('off')
        plt.title(f"{os.path.basename(img_path)} ({num_detections} detections)")
        plt.show()

        vis_save_path = os.path.join(
            save_dir,
            f"{os.path.splitext(os.path.basename(img_path))[0]}_det.png"
        )
        cv2.imwrite(vis_save_path, cv2.cvtColor(visulize_img, cv2.COLOR_RGB2BGR))
        print(f"Saved visualization to: {vis_save_path}")

    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        detection_results.append((os.path.basename(img_path), 0))


Output hidden; open in https://colab.research.google.com to view.

In [18]:

df = pd.DataFrame(detection_results, columns=["filename", "num_detections"])
out_csv = os.path.join("/content/drive/MyDrive/FISHIAL", "detection_counts.csv")
os.makedirs(os.path.dirname(out_csv), exist_ok=True)
df.to_csv(out_csv, index=False, encoding="utf-8-sig")

print(f"\n Saved detection counts to {out_csv}")


 Saved detection counts to /content/drive/MyDrive/FISHIAL/detection_counts.csv
